# ¿Quién sobrevivió al Titanic?

En este ejercicio, utilizaremos los datos Titanic para predecir si un pasajero sobrevivió o no al naufragio.
Veremos diferentes formas de hacerlo; con varios grados de automatismo.

## Human Learn

Human Learn es una librería que nos permite combinar el aprendizaje automático con el conocimiento humano de una manera simple.

In [ ]:
# Cargamos las librerías y los datos

In [ ]:
# Instalamos la librería
%pip install git+https://github.com/koaning/human-learn.git

In [ ]:
# Cargamos el Dataset
from hulearn.datasets import load_titanic

In [ ]:
df = load_titanic(as_frame=True)
df

In [ ]:
from sklearn.model_selection import train_test_split

# Preparamos los datos: separamos las variables explicativas de la variable de destino y creamos los conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['name', 'survived'], axis=1), df['survived'], test_size=0.25, random_state=42)

### Clasificación con una función

En este primer ejemplo, utilizaremos una función para clasificar a los pasajeros. Es una función muy simple que clasifica a los pasajeros en función del precio del billete. El enfoque es que los pasajeros que más pagaban tenían más probabilidades de sobrevivir.

In [ ]:
# FunctionClassifier nos permite crear un clasificador a partir de una función
from hulearn.classification import FunctionClassifier

In [ ]:
# Usaremos el FunctionClassifier Para crear un clasificador que use el precio del boleto para predecir si un pasajero sobrevivió o no

def classificador_preu_bitllet(df, preu_minim=20):
    """
    Clasificar a los pasajeros según el precio del billete
    El enfoque es que los pasajeros que más pagaban tenían más probabilidades de sobrevivir.
    El valor predeterminado es 20 pero podemos cambiarlo
    """
    return (df['fare'] > preu_minim).astype(int)


model = FunctionClassifier(classificador_preu_bitllet)

# Preparem el classificador
model.fit(X_train, y_train)

### Evaluamos el clasificador

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model.predict(X_test)))

El rendimiento del clasificador es bastante malo. Seguramente el precio del boleto no es una buena variable para predecir si un pasajero sobrevivió o no. Aun así, intentaremos mejorar su rendimiento de refinación el parámetro de precio_minim.

Usaremos GridSearchCV para encontrar el mejor valor para Price_Minim. GridSearchCV nos permite hacer una búsqueda de cuadrícula para encontrar los mejores parámetros, la puntuación será dada por la exactitud (Accuracy), la precisión (Precision) y la sensibilidad (Recall).

- **Accuracy**: ¿Cuántas predicciones son correctas?
- **Precision**: ¿Cuántas predicciones positivas son correctas?
- **Recall**: ¿Cuántas predicciones positivas son correctas de todos los aspectos positivos?

In [ ]:
import numpy as np
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

# Creamos un clasificador con un precio mínimo de 20
mod = FunctionClassifier(classificador_preu_bitllet, preu_minim=20)

# El objeto GridSearchCV nos permite hacer una búsqueda de cuadrícula para encontrar los mejores parámetros.
# En este caso buscaremos el mejor precio mínimo para clasificar a los pasajeros
grid = GridSearchCV(mod,
                    cv=2,
                    param_grid={'preu_minim': np.linspace(0, 100, 30)},
                    scoring={'accuracy': make_scorer(accuracy_score),
                             'precision': make_scorer(precision_score),
                             'recall': make_scorer(recall_score)},
                    refit='accuracy')
grid.fit(X_train, y_train)

print(classification_report(y_test, grid.predict(X_test)))

In [ ]:
# Podemos ver cuál era el mejor precio mínimo
grid.best_params_

### Exploración interactiva

Ahora que hemos visto que el precio del boleto no es una buena variable para predecir si un pasajero sobrevivió o no, podemos explorar los datos para encontrar una mejor variable.

Para hacer esto, utilizaremos la función Interactivecharts.Esta función nos permitirá explorar los datos de manera interactiva.En este caso, utilizaremos la función Parallel_Coordinates para ver cómo se distribuyen los datos en función de las variables explicativas.

In [ ]:
from hulearn.experimental.interactive import parallel_coordinates

parallel_coordinates(df, label="survived", height=200)

Basado en la hipótesis de que las mujeres y los niños tenían más probabilidades de sobrevivir, podemos explorar datos para ver si es cierto y parece sí (contar con las diferencias de clase social): si solo dejamos mujeres y niños en primera y segunda clase, la mayoría sobrevivió.

Entonces podemos crear un clasificador que use el sexo, la edad y la clase del pasajero para predecir si sobrevivió o no.

In [ ]:
def classificador_edat_sexe_classe(df, edat=12):
    """
    Clasificar a los pasajeros según el sexo, la edad y la clase de pasajeros
    El enfoque es que las mujeres y los niños tenían más probabilidades de sobrevivir
    """
    regla_dones = (df['pclass'] < 3.0) & (df['sex'] == "female")
    regla_nens = (df['pclass'] < 3.0) & (df['age'] <= edat)
    return regla_dones | regla_nens


mod = FunctionClassifier(classificador_edat_sexe_classe)
mod.fit(X_train, y_train)
mod.score(X_test, y_test)

In [ ]:
grid = GridSearchCV(mod,
                    cv=2,
                    param_grid={'edat': np.linspace(0, 50, 50)},
                    scoring={'accuracy': make_scorer(accuracy_score),
                             'precision': make_scorer(precision_score),
                             'recall': make_scorer(recall_score)},
                    refit='accuracy')
grid.fit(X_train, y_train)

print(grid.best_params_)
predict = grid.predict(X_train)

print(predict)
print(classification_report(y_test, grid.predict(X_test)))

Los números son lo mucho mejores que los anteriores. Por lo tanto, podemos usar este clasificador para hacer predicciones en nuevos pasajeros.

## FIGS

FIGS Es una librería que nos permite crear reglas para clasificar los datos.En este caso, utilizaremos los datos Titanic para crear reglas que nos permitan predecir si un pasajero sobrevivió o no.

In [ ]:
# Instalamos la librería
%pip install imodels

In [ ]:
# Cargamos la librería
from imodels import FIGSClassifier

In [ ]:
# Preparamos los datos
X_skope = df.drop(columns=['name', 'sibsp'])
X_skope['sex'] = X_skope['sex'].replace(['female', 'male'], [0, 1])

X_train, X_test, y_train, y_test = train_test_split(
    X_skope.drop(['survived'], axis=1), X_skope['survived'], test_size=0.25, random_state=42)

In [ ]:
# Creamos el clasificador
clf = FIGSClassifier(max_rules=4)
# Entrenamos el clasificador
clf.fit(X_train, y_train)

In [ ]:
# Podem veure el rendiment del classificador
print(classification_report(y_test, clf.predict(X_test)))

Podemos ver que las reglas creadas por el clasificador tienen un rendimiento similar al clasificador que hemos creado con Human Learn y los conceptos que hemos creado con Human Learn son más fáciles de interpretar y podemos continuar agregando reglas para mejorar el rendimiento del clasificador (por ejemplo, podríamos incorporar la variable `fare`(tarifa)).